# Installing Hub

In [ ]:
!pip3 install hub --quiet

# Run below cells and restart the runtime
# if you are running it in colab
# import os
# os.kill(os.getpid(), 9) 

# Download raw dataset

In [ ]:
from IPython.display import clear_output

In [ ]:
# Download Dataset here
!wget https://www.cse.cuhk.edu.hk/leojia/projects/hsaliency/data/CSSD/images.zip
!wget https://www.cse.cuhk.edu.hk/leojia/projects/hsaliency/data/CSSD/ground_truth_mask.zip
    
!unzip ground_truth_mask.zip 
!unzip images.zip

clear_output()

In [ ]:
import numpy as np 
import os
import time
from tqdm import tqdm
import cv2

# Creating dataset on hub

**Activeloop API** : https://docs.activeloop.ai/api-basics

In [ ]:
import hub

# Login to ActiveLoop

%env BUGGER_OFF=True
!activeloop login -u username -p password
!activeloop reporting --off

In [ ]:
image_path

In [ ]:
cv2.imread(image_path).shape

In [ ]:
cv2.imread(mask_path).shape

In [ ]:
mask.shape

In [ ]:
# hubpath = './cssd-hub'
hubpath = 'hub://activeloop/cssd'
# hub.delete(hubpath, large_ok=True)
ds = hub.dataset(hubpath)

start = time.time()

with ds:
    ds.create_tensor('images', htype='image', sample_compression='jpg')
    ds.create_tensor('masks', htype='binary_mask', sample_compression='lz4')
    
    for image in tqdm(os.listdir('images/')):
        file = image.split('.')[0]
        image_path = os.path.join('images', file+'.jpg')
        mask_path = os.path.join('ground_truth_mask', file+'.png')
        
        img_pil = Image.open(mask_path).convert('L')
        mask = np.array(img_pil, dtype='uint8')
        m, n = mask.shape
        mask = mask.reshape(m, n, 1)
            
        ds.append({
            'images' : hub.read(image_path),
            'masks' : mask.astype('bool')
        }, skip_ok=True)
        
stop = time.time()
print(f"Time elapsed uploading : {stop-start}")

# Testing dataset from hub

In [ ]:
hubpath = 'hub://activeloop/cssd'
ds = hub.load(hubpath)
print(ds.tensors)

In [ ]:
print(f"Image Shape: {ds.images[0].shape}")
print(f"Masks Shape: {ds.masks[0].shape}")